<a href="https://colab.research.google.com/github/Jyoti-Hajjargi/homomorphic_api.py/blob/main/homomorphic_api_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# STEP 1 — Install Required Libraries



In [ ]:
from fastapi import FastAPI
app = FastAPI()

@app.get("/")
def home():
    return {"message": "Hello World"}

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio pyfhel
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

# STEP 2 — Create FastAPI Server File (main.py)

In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
import time
from Pyfhel import Pyfhel, PyCtxt

app = FastAPI()

# Initialize the Homomorphic Encryption context
HE = Pyfhel()
HE.contextGen(scheme='BFV', n=2**14, t=65537) # Corrected parameters
HE.keyGen()

class Number(BaseModel):
    value: float

class EncryptedData(BaseModel):
    encrypted: str


# ----------------------- ENCRYPT ENDPOINT -----------------------
@app.post("/encrypt")
def encrypt_number(data: Number):
    encrypted = HE.encryptFrac(data.value)
    encrypted_str = encrypted.to_bytes().hex()
    return {"encrypted": encrypted_str}


# ----------------------- DECRYPT ENDPOINT -----------------------
@app.post("/decrypt")
def decrypt_number(data: EncryptedData):
    encrypted_bytes = bytes.fromhex(data.encrypted)
    ctxt = PyCtxt()
    ctxt.from_bytes(encrypted_bytes, HE)
    decrypted = HE.decryptFrac(ctxt)
    return {"decrypted": decrypted}


# ----------------------- ADD ENCRYPTED NUMBERS -----------------------
@app.post("/add")
def add_numbers(a: EncryptedData, b: EncryptedData):
    ctxt_a = PyCtxt()
    ctxt_b = PyCtxt()

    ctxt_a.from_bytes(bytes.fromhex(a.encrypted), HE)
    ctxt_b.from_bytes(bytes.fromhex(b.encrypted), HE)

    result = ctxt_a + ctxt_b
    result_hex = result.to_bytes().hex()
    return {"result": result_hex}


# ----------------------- PERFORMANCE TEST -----------------------
@app.get("/performance")
def performance_test():
    start = time.time()

    a = HE.encryptFrac(10.5)
    b = HE.encryptFrac(20.3)
    c = a + b

    end = time.time()
    return {
        "status": "OK",
        "encrypted_computation_time_seconds": end - start
    }

Overwriting main.py


# STEP 3 — Run FastAPI Server in Background

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
import asyncio # Import asyncio

# Run API
# uvicorn.run("main:app", host="0.0.0.0", port=8000)

# Manually configure and run Uvicorn to avoid the loop_factory issue with nest_asyncio
config = uvicorn.Config("main:app", host="0.0.0.0", port=8000, loop="asyncio")
server = uvicorn.Server(config);

# Use asyncio.run() directly, which nest_asyncio has patched
asyncio.run(server.serve())

INFO:     Started server process [3601]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


# STEP 4 — Expose API using LocalTunnel (Alternative to ngrok)

In [ ]:
!lt --port 8000